#  Setup, connectivity ..

In [ ]:
import os

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


# Graph maintenance ..

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE A GRAPH

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

# Actually doing the import ..

In [ ]:
#  COPY FILES UP TO S3 ..
#
#     https://console.cloud.google.com/storage/browser/farrell-data-bucket
#

gsutil cp   /opt/kg/cluster_P/workbook/26_AllThingsMongo/08_CsvData/20_movies.txt                          gs://farrell-data-bucket/mdb/20_movies.txt
gsutil cp   /opt/kg/cluster_P/workbook/26_AllThingsMongo/08_CsvData/21_users.txt                           gs://farrell-data-bucket/mdb/21_users.txt
gsutil cp   /opt/kg/cluster_P/workbook/26_AllThingsMongo/08_CsvData/30_Edge_Commented.txt                  gs://farrell-data-bucket/mdb/30_Edge_Commented.txt

gsutil cp   /opt/kg/cluster_P/workbook/26_AllThingsMongo/08_CsvData/10_nodes.txt                           gs://farrell-data-bucket/mdb/10_nodes.txt
gsutil cp   /opt/kg/cluster_P/workbook/26_AllThingsMongo/08_CsvData/11_edges.txt                           gs://farrell-data-bucket/mdb/11_edges.txt
   

In [ ]:
# ##################################################################

In [ ]:
#  Import data using CSV ..

l_inp="gs://farrell-bucket/mongodb/10_nodes.txt"
l_enp="gs://farrell-bucket/mongodb/11_edges.txt"
   #
l_dir="gs://farrell-bucket/mongodb"


import_data.csv(
   my_graph,
      #
   input_node_path     = l_inp,
   input_edge_path     = l_enp,
   input_dir           = l_dir,
      #
   data_delimiter      = ",",
   schema_delimiter    = ",",
   files_have_headers  = True,
   )


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:

l_result = my_graph.query("""

   MATCH (n)  - [ r ] ->  (m )
   RETURN n, m, r
   LIMIT 100
   
   """, contextualize=True)

l_result.view()
